In [1]:
from IPython.core.display import HTML, display
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
# Specify the path to the fastai directory
lib_PATH = '/home/ubuntu/fastai/fastai'   # Update this
!cd {lib_PATH}
# Create sym-link to the fastai library
!ln -s {lib_PATH} ./

ln: failed to create symbolic link './fastai': File exists


In [4]:
# Load required modules
from fastai.imports import *
from fastai.structured import *
from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestClassifier
from IPython.display import display
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from dateutil.relativedelta import relativedelta

# Load Cleaned Data

In [ ]:
train = pd.read_feather('/home/ubuntu/wsmd_music_rec/data/tmp/wsdm_train')
test = pd.read_feather('/home/ubuntu/wsmd_music_rec/data/tmp/wsdm_test')

In [5]:
# Ratio of 1 and 0 for target var
 





In [6]:
test.song_year.replace([np.inf, -np.inf], np.nan, inplace=True)
test.song_year.fillna(-1,inplace=True)
test.song_year = test.song_year.astype('int64')

In [ ]:
train.columns[train.isnull().any()]

In [7]:
df, y, nas = proc_df(train, 'target')    # proc_df deals with na values

In [ ]:
df.columns[df.isnull().any()]     # no nas left

# Split into Training + Validation Sets

In [ ]:
fig=plt.figure(figsize=(18, 16))
test.song_year.value_counts().plot.barh()

In [8]:
def split_vals(a, n): return a[:n].copy(), a[n:].copy()
# https://www.kaggle.com/kamilkk/i-have-to-say-this
# temporally ordered data - set val. set = test set's length
len_valid = len(test)
n_trn = len(df) - len_valid
X_train, X_valid = split_vals(df, n_trn)
y_train, y_valid = split_vals(y, n_trn)

X_train.shape, y_train.shape, X_valid.shape

((4820628, 35), (4820628,), (2556790, 35))

# Train the Random Forest

In [9]:
def rmse(x,y): return math.sqrt(((x-y)**2).mean())

def print_score(m):
    auc_trn   = roc_auc_score(y_train, m.predict(X_train))
    auc_valid = roc_auc_score(y_valid, m.predict(X_valid))
    res = [auc_trn, auc_valid, rmse(m.predict(X_train), y_train), rmse(m.predict(X_valid), y_valid), 
           m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print('auc_trn, auc_valid, RMSE - train, RMSE - valid, R^2 - train, R^2 - valid')
    print(res)
    
def print_score_final(m):
    auc_trn = roc_auc_score(y, m.predict(df))
    res = [auc_trn, rmse(m.predict(df), y), m.score(df, y)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print('auc_trn, RMSE - train, R^2 - train')
    print(res)

In [ ]:
m = RandomForestClassifier(n_estimators=50, min_samples_leaf=20, n_jobs=-1, verbose=1)
%time m.fit(X_train, y_train)
print_score(m)

# Retrain Model Using All Data

In [10]:
m_final = RandomForestClassifier(n_estimators=200, min_samples_leaf=200, n_jobs=-1, verbose=1)
%time m_final.fit(df, y)

[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed: 12.8min


CPU times: user 1h 48min 43s, sys: 45.5 s, total: 1h 49min 29s
Wall time: 13min 50s


[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed: 13.8min finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=200, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=-1,
            oob_score=False, random_state=None, verbose=1,
            warm_start=False)

In [ ]:
print_score_final(m_final)

# Make Predictions

In [11]:
test_df, _, _ = proc_df(test, 'id')

In [12]:
%time predictions = m_final.predict(test_df)

[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    3.7s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:   18.3s


CPU times: user 2min 34s, sys: 2.57 s, total: 2min 36s
Wall time: 20.6 s


[Parallel(n_jobs=8)]: Done 200 out of 200 | elapsed:   19.8s finished


In [13]:
output = pd.concat([test['id'], pd.DataFrame(predictions)], axis=1)
output.columns = ['id', 'target']   #Changing column names
output.to_csv('predicted.csv', index=False)